In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from sklearn.model_selection import train_test_split
import os
import matplotlib.pyplot as plt
from scipy import stats
import ast 

In [2]:
categories = ['banana','beach','car','church','lighthouse','mountain','mug','plane']
path='../Analysis/human_rating_bycat'

In [3]:
for i, cate in enumerate(categories):
    col = [cate+'_'+str(i+1) for i in range(16)]
    new = pd.read_csv(path+'/'+cate+'.csv')
    new.columns = col
    if i == 0:
        old = new
    else:
        old = pd.concat([old,new],axis=1)
df = old

In [12]:
df

,banana_1,banana_2,banana_3,banana_4,banana_5,banana_6,banana_7,banana_8,banana_9,banana_10,...,plane_7,plane_8,plane_9,plane_10,plane_11,plane_12,plane_13,plane_14,plane_15,plane_16
0,7,5,6,5,5,7,7,6,6,4,...,7,3,7,7,2,5,6,4,4,7
1,7,6,7,6,6,6,6,6,6,6,...,7,3,4,7,3,6,5,5,3,7
2,7,7,7,6,6,7,7,7,7,7,...,7,4,6,7,4,5,6,5,5,7
3,7,7,7,2,3,7,7,5,5,4,...,7,2,4,7,2,3,3,1,2,7
4,7,6,7,5,5,7,7,6,6,5,...,6,4,5,6,4,4,5,3,4,7
5,5,3,3,5,6,7,4,7,5,6,...,7,2,5,7,4,5,6,4,3,6
6,7,7,7,7,7,7,7,7,7,7,...,7,2,7,7,2,4,4,7,5,7
7,6,6,5,1,2,6,4,4,7,2,...,6,4,4,5,3,3,5,1,3,5
8,5,4,4,2,2,6,5,6,5,3,...,4,6,4,5,6,3,6,2,5,3
9,7,7,7,6,6,7,7,7,7,6,...,7,4,5,7,3,5,5,2,3,7


(Test) Use quantiles to remove outliers 

In [4]:
quantile_low = df.quantile(0.1)
quantile_high = df.quantile(0.9)
iqr = quantile_high - quantile_low

In [5]:
outliers = (df < quantile_low) | (df > quantile_high)
df_new = df[outliers.sum(axis=1) <= 12]

Compute 7 scorse and sort images by order in survey

In [6]:
# alexnet_df=pd.read_csv('alexnet_scores_wordnet_id_closest_label.csv',index_col=0)
# alexnet_df['typicality_score'] = np.round(alexnet_df['closest_prob'] * 100,2)
# alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
# alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

# alexnet_df=pd.read_csv('alexnet_scores_wordnet_id_matched_label.csv',index_col=0)
# alexnet_df['typicality_score'] = np.round(alexnet_df['matched_prob'] * 100,2)
# alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
# alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

alexnet_df=pd.read_csv('alexnet_scores_final.csv',index_col=0)
alexnet_df['typicality_score'] = np.round(alexnet_df['prob'] * 100,2)
alexnet_df['7_score']= np.round(alexnet_df['typicality_score']*7/100,2)
alexnet_df['type'] = alexnet_df['type'].apply(lambda x: x.lower())

In [7]:
alexnet_df['number'] = alexnet_df['dir'].apply(lambda x: int(x.split('\\')[-1].split('.')[0]))
alexnet_df = alexnet_df.sort_values(['type','number']).reset_index(drop=True)

In [ ]:
# alexnet_df['readable_labels'].apply(lambda x: ast.literal_eval(x)[list(ast.literal_eval(x).keys())[0]])

# Correlation with human ratings

Changes from original script: 
1. Use spearman rank correlation instead of pearson
2. Compute average human ratings first, and then compute correlation with neural net typicality scores

without dropping outliers

In [8]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    human_ratings = df[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
#     net_scores = net_ratings['readable_labels'].apply(lambda x: ast.literal_eval(x)[list(ast.literal_eval(x).keys())[0]]).values
#     net_scores = net_ratings['7_score'].values # both get worse scores

    net_scores = net_ratings['typicality_score'].values
    
    coef, p=stats.spearmanr(human_scores, net_scores)
#     coef, p=stats.pearsonr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.6254600663118389
beach:  0.4323529411764706
car:  0.4970588235294117
church:  0.3649743445772613
lighthouse:  0.5735294117647058
mountain:  0.5518765290986815
mug:  0.4153170925333297
plane:  0.6975719327807334


In [9]:
print(np.mean(list(final_corr.values())))

0.5197676427215542


after dropping outliers

In [10]:
final_corr = dict()
for t in np.unique(alexnet_df['type']):
    col_names = [t + '_' + str(i+1) for i in range(16)]
    
    human_ratings = df_new[col_names] 
    net_ratings = alexnet_df[alexnet_df['type'] == t] 
    
    human_scores = human_ratings.mean().values
    net_scores = net_ratings['7_score'].values
    coef, p=stats.spearmanr(human_scores, net_scores)
    final_corr[t]=coef
    print(f'{t}: ', final_corr[t])

banana:  0.596026651426576
beach:  0.4163597298539862
car:  0.5419740781995225
church:  0.3784977908689249
lighthouse:  0.5537561233777729
mountain:  0.5979393019752514
mug:  0.3867168149822578
plane:  0.6642082700076948


In [11]:
print(np.mean(list(final_corr.values())))

0.5169348450864983


In [17]:
alexnet_df['human_score'] = np.array(df.apply(np.mean,axis=0))

In [18]:
alexnet_df.to_csv('alexnet_result.csv')